In [45]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.feature_extraction import text  # 기본 영어 불용어를 가져오기 위함

In [46]:
# CSV 파일에서 데이터 불러오기
apps_data = pd.read_csv('cleaned_apps_data.csv')

In [47]:
# 2. 앱 이름을 불용어 리스트로 처리
app_names = apps_data['Name'].apply(lambda x: x.lower()).tolist()  # 앱 이름을 소문자로 변환하여 리스트로 수집


In [48]:
# 3. 기본 불용어와 앱 이름 결합
stop_words = list(text.ENGLISH_STOP_WORDS.union(app_names))  # frozenset을 리스트로 변환


In [49]:
# 2. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence-BERT 모델
embeddings = model.encode(apps_data['Cleaned_Description'].values)  # 앱 설명에 대한 임베딩 생성


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [58]:
# 3. K-Means 클러스터링 수행
n_clusters = 5  # 클러스터 수 설정
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [59]:
# 4. 클러스터 결과를 데이터프레임에 추가
apps_data['Cluster'] = clusters



In [60]:
# 7. TF-IDF 기반 주요 키워드 추출 함수
# 앱 이름을 stop_words로 추가하여 TF-IDF에서 제외
vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=1000)  # 앱 이름을 불용어로 처리
tfidf_matrix = vectorizer.fit_transform(apps_data['Cleaned_Description'])

def get_top_keywords(tfidf_matrix, clusters, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords

c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['309', '4x', 'adblocker', 'billionaire', 'blocker', 'booster', 'carrera', 'coupang', 'delivery', 'digital', 'eats', 'elegant', 'face', 'fas', 'faster', 'filmhwa', 'filter', 'food', 'game', 'hwa', 'hybrid', 'illuminator', 'like', 'lite', 'md242', 'min', 'movies', 'news', 'play', 'plus', 'pro', 'samsung', 'shop', 'tag', 'temu', 'tiktok', 'today', 'tv', 'unicorn', 'watch', 'wfp', '건강보험증', '경기도', '기회소득', '기후행동', '모바일'] not in stop_words.
  warnings.warn(


In [61]:
# 6. 클러스터별 상위 5개 키워드 추출
top_keywords_per_cluster = get_top_keywords(tfidf_matrix, clusters, top_n=5)



In [62]:
# 7. 키워드 기반으로 상위 카테고리 이름 생성
def generate_category_name(keywords):
    return " & ".join(keywords[:2])  # 상위 2개의 키워드를 연결하여 이름 생성



In [63]:
# 8. 클러스터별 상위 카테고리 이름 생성 및 출력
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    print(f"Cluster {cluster_num} Name: {category_name}")

Cluster 0 Name: tv & ads
Cluster 1 Name: news & receive
Cluster 2 Name: insurance & pay
Cluster 3 Name: watch & face
Cluster 4 Name: game & lite


In [64]:
# 9. 클러스터 이름을 데이터프레임에 추가
for cluster_num, keywords in enumerate(top_keywords_per_cluster):
    category_name = generate_category_name(keywords)
    apps_data.loc[apps_data['Cluster'] == cluster_num, 'Super_Category'] = category_name


In [23]:
# 10. 결과 확인
print(apps_data[['Category', 'Cleaned_Description', 'Cluster', 'Super_Category']])

                   Category  \
0          Health & Fitness   
1   Video Players & Editors   
2                  Shopping   
3                    Social   
4                 Lifestyle   
5             Entertainment   
6          News & Magazines   
7                   Finance   
8              Food & Drink   
9             Entertainment   
10          Personalization   
11                    Tools   
12          Personalization   
13          Personalization   
14              Photography   
15             Productivity   
16          Personalization   

                                          Description  Cluster  Super_Category  
0   1. The mobile health insurance card app is a p...        0    watch & face  
1   TikTok Lite is a global video community that i...        2  coupang & game  
2   Shop on Temu for exclusive offers. \n\nNo matt...        1     news & temu  
3   Say more with Threads — Instagram’s text-based...        1     news & temu  
4   Through carbon neutrality activi